<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 13:20:31
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -20.79 K (-0.15%)
Current PnL: -13.77 L (-9.59%)
CY Booked + Current PnL: -7.03 L (-4.9%)
-------------------
Total profit:  4.34 L
Total loss:  -18.11 L
-------------------
Total Booked + Current PnL: 19.95 L (17.12%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.19 L (53.03%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,79.17,74255.0,39806.0,141.0,-0.19,115.55,0.19,115.97,NaN,282.31,0.55,13.55,AR,ATH,ELECTRICAL
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16070.0,-33523.0,2446.0,-3.37,-67.60,15.22,-62.66,NaN,-13.71,0.12,7.77,XY24,ATH,MISC
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,90.62,133839.0,-8640.0,9262.0,1.10,-6.06,6.92,0.44,243.0,-0.93,0.98,57.97,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,29.17,102950.0,24588.0,9441.0,-0.01,31.38,9.17,43.43,77.0,2.60,0.76,38.37,XY24,NTT,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42267.0,1012.0,9793.0,-1.60,2.45,23.17,26.19,1.0,0.10,0.31,0.00,XY24,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,98.96,140858.0,-984.0,57555.0,1.64,-0.69,40.86,39.88,259.0,-0.02,1.04,42.75,XY24,NTT,POWER
15,BLUESTARCO,1646.70,2337.55,-12.78,H-SC,83.33,163560.0,-1110.0,70200.0,0.71,-0.67,42.92,41.95,121.0,-0.02,1.20,6.74,X40N,ATH,AC
29,DIXON,14367.00,18940.15,-37.49,H-MC,25.00,115160.0,224.0,36356.0,-3.92,0.19,31.57,31.83,99.0,0.01,0.85,31.99,X40N,ATH,IT
60,NATIONALUM,189.63,247.44,-50.15,H-MC,64.58,100589.0,85.0,30559.0,-1.51,0.08,30.38,30.49,97.0,0.00,0.74,32.52,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,SYMPHONY,1306.42,1306.00,-31.65,M-SC,6.25,145790.0,-25351.0,25295.0,3.51,-14.81,17.35,-0.03,189.0,-1.00,1.07,4.78,OX40N,NTT,DURABLES
5,APOLLOHOSP,6904.43,8285.00,-17.74,H-LC,76.04,112072.0,8506.0,12205.0,3.17,8.21,10.89,20.00,30.0,0.70,0.82,23.44,X40N,BTT,HEALTHCARE
74,SIEMENS,4406.90,7969.85,-1.77,X-LC,43.75,83080.0,-27092.0,116162.0,2.21,-24.59,139.82,80.85,14.0,-0.23,0.61,22.36,AR,ATH,ELECTRICAL
47,INDUSINDBK,1354.50,1800.00,87.72,L-MC,85.42,50756.0,-26450.0,51842.0,2.10,-34.26,102.14,32.89,253.0,-0.51,0.37,39.78,XR,NTT,BANKS
54,KPIGREEN,497.21,732.16,29.50,H-SC,72.92,132350.0,7053.0,52159.0,2.05,5.63,39.41,47.25,139.0,0.14,0.97,65.58,MH,ATH,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,DIXON,14367.00,18940.15,-37.49,H-MC,25.00,115160.0,224.0,36356.0,-3.92,0.19,31.57,31.83,99.0,0.01,0.85,31.99,X40N,ATH,IT
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16070.0,-33523.0,2446.0,-3.37,-67.60,15.22,-62.66,NaN,-13.71,0.12,7.77,XY24,ATH,MISC
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,96.88,99403.0,-1560.0,96093.0,-2.38,-1.55,96.67,93.63,64.0,-0.02,0.73,48.06,SR,ATH,CHEMICALS
1,ABB,5551.76,7934.00,-35.00,H-LC,32.81,231972.0,15453.0,77455.0,-2.18,7.14,33.39,42.91,28.0,0.20,1.70,21.25,AR,NTT,ELECTRICAL
61,NESTLEIND,2268.60,2755.00,-10.95,X-LC,38.54,282309.0,16883.0,40031.0,-2.13,6.36,14.18,21.44,9.0,0.42,2.07,12.95,XY25,NTT,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,90.62,133839.0,-8640.0,9262.0,1.10,-6.06,6.92,0.44,243.0,-0.93,0.98,57.97,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-216.05,H-SC,97.92,83714.0,-11602.0,11645.0,1.94,-12.17,13.91,0.04,154.0,-1.00,0.62,65.12,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-31.65,M-SC,6.25,145790.0,-25351.0,25295.0,3.51,-14.81,17.35,-0.03,189.0,-1.00,1.07,4.78,OX40N,NTT,DURABLES
81,TATAELXSI,7332.28,7332.0,-16.81,X-MC,23.96,87255.0,-15397.0,15392.0,-1.23,-15.00,17.64,-0.00,58.0,-1.00,0.64,32.64,OX40N,NTT,IT
46,INDIGOPNTS,1407.73,1408.0,123.21,H-SC,87.50,146456.0,-28103.0,28134.0,1.63,-16.10,19.21,0.02,127.0,-1.00,1.08,27.00,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42267.0,1012.0,9793.0,-1.60,2.45,23.17,26.19,1.0,0.10,0.31,0.00,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.74,X-LC,42.71,268320.0,-27614.0,121522.0,-0.64,-9.33,45.29,31.73,1.0,-0.23,1.97,7.40,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,59.38,287796.0,26195.0,119435.0,0.37,10.01,41.50,55.67,2.0,0.22,2.12,16.73,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,20.83,257141.0,-13206.0,47828.0,0.06,-4.88,18.60,12.81,6.0,-0.28,1.89,7.54,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-10.95,X-LC,38.54,282309.0,16883.0,40031.0,-2.13,6.36,14.18,21.44,9.0,0.42,2.07,12.95,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,39.58,174768.0,33928.0,10556.0,0.07,24.09,6.04,31.59,89.0,3.21,1.28,51.81,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,29.17,102950.0,24588.0,9441.0,-0.01,31.38,9.17,43.43,77.0,2.60,0.76,38.37,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,79.17,74255.0,39806.0,141.0,-0.19,115.55,0.19,115.97,NaN,282.31,0.55,13.55,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,90.28,M-MC,48.96,228131.0,3169.0,164482.0,0.37,1.41,72.10,74.53,185.0,0.02,1.68,32.45,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,29.50,H-SC,72.92,132350.0,7053.0,52159.0,2.05,5.63,39.41,47.25,139.0,0.14,0.97,65.58,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.65,H-SC,53.12,118656.0,16652.0,28371.0,-1.22,16.32,23.91,44.14,124.0,0.59,0.87,35.22,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,29.17,102950.0,24588.0,9441.0,-0.01,31.38,9.17,43.43,77.0,2.60,0.76,38.37,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,39.58,174768.0,33928.0,10556.0,0.07,24.09,6.04,31.59,89.0,3.21,1.28,51.81,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,79.17,74255.0,39806.0,141.0,-0.19,115.55,0.19,115.97,NaN,282.31,0.55,13.55,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.18,M-SC,91.67,203742.0,32121.0,129947.0,-1.10,18.72,63.78,94.43,198.0,0.25,1.50,50.37,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.16,29.50,H-SC,72.92,132350.0,7053.0,52159.0,2.05,5.63,39.41,47.25,139.0,0.14,0.97,65.58,MH,ATH,POWER
95,WIPRO,243.46,420.00,-4.22,M-LC,73.96,164269.0,13324.0,96130.0,-0.39,8.83,58.52,72.51,69.0,0.14,1.21,13.15,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,92.71,137010.0,13674.0,120089.0,-0.50,11.09,87.65,108.45,122.0,0.11,1.01,35.92,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,71789.0,-41760.0,81811.0,-0.16,-36.78,113.96,35.27,261.0,-0.51,0.53,75.70,XR,NTT,HOTELS
32,ENRIN,1377.95,2975.90,NaN,NaN,79.17,74255.0,39806.0,141.0,-0.19,115.55,0.19,115.97,NaN,282.31,0.55,13.55,AR,ATH,ELECTRICAL
58,MASFIN,326.60,402.39,-18.25,H-SC,65.62,92715.0,-5265.0,28000.0,0.32,-5.37,30.20,23.21,137.0,-0.19,0.68,34.47,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,96.88,99403.0,-1560.0,96093.0,-2.38,-1.55,96.67,93.63,64.0,-0.02,0.73,48.06,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,29.17,102950.0,24588.0,9441.0,-0.01,31.38,9.17,43.43,77.0,2.60,0.76,38.37,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,71789.0,-41760.0,81811.0,-0.16,-36.78,113.96,35.27,261.0,-0.51,0.53,75.70,XR,NTT,HOTELS
52,JPPOWER,18.73,26.20,-30.08,L-SC,98.96,140858.0,-984.0,57555.0,1.64,-0.69,40.86,39.88,259.0,-0.02,1.04,42.75,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-216.05,H-SC,97.92,83714.0,-11602.0,11645.0,1.94,-12.17,13.91,0.04,154.0,-1.00,0.62,65.12,OX40N,NTT,MISC
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,96.88,99403.0,-1560.0,96093.0,-2.38,-1.55,96.67,93.63,64.0,-0.02,0.73,48.06,SR,ATH,CHEMICALS
51,JIOFIN,310.58,387.00,7.94,H-LC,95.83,231350.0,13944.0,39561.0,1.15,6.41,17.10,24.61,95.0,0.35,1.70,64.49,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.85
1,25,41.94
2,50,70.43


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.54
LC    34.11
MC    21.39
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.54
X40      12.25
XY25     12.03
X40N     11.71
XR       10.18
AR        8.27
OX40N     8.23
MH        1.71
X5K       1.48
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.65
M-SC    16.04
H-LC    14.15
X-LC    12.21
H-MC     9.45
X-MC     7.51
M-LC     6.69
X-SC     4.36
M-MC     4.06
L-SC     2.49
L-LC     1.06
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.10,-1.86,35.92
IT,12.06,-10.73,65.14
BANKS,7.99,-4.69,47.40
FINANCE,7.95,-11.44,51.78
MISC,6.66,-15.00,68.25
PAINTS,5.67,-13.39,35.74
ELECTRICAL,5.49,1.36,44.71
HEALTHCARE,4.50,-5.18,35.90
AUTO,4.32,-14.92,65.45


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2714030.0,27
XR,1075087.0,13
AR,1017962.0,10
X40,654628.0,9
X40N,568617.0,11
XY25,480807.0,8
OX40N,311209.0,11
SR,177586.0,2
MH,82718.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1973986.0,20
M-SC,1761718.0,21
H-LC,613231.0,12
H-MC,549500.0,8
X-LC,513321.0,7
X-MC,430739.0,6
M-MC,418358.0,3
X-SC,325446.0,5
M-LC,263524.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      859534.0      6
M-SC       XY24      814533.0      7
H-SC       AR        519008.0      3
M-SC       XR        333905.0      4
M-MC       XY24      309737.0      2
H-MC       XY24      300614.0      3
X-LC       X40       240957.0      2
H-SC       X40N      236115.0      4
           XR        217654.0      3
H-LC       X40N      200987.0      4
M-SC       AR        181839.0      2
           OX40N     165971.0      5
H-LC       X40       159000.0      3
M-LC       XY24      156857.0      3
X-MC       X40       152939.0      2
L-SC       XR        143212.0      2
X-SC       XR        123723.0      1
X-LC       XY25      119416.0      3
           AR        116162.0      1
M-SC       XY25      111630.0      1
M-MC       XR        108621.0      1
X-MC       XY25      105935.0      1
           XY24       98100.0      1
M-LC       XR         96130.0      1
X-SC       SR         96093.0      1
H-SC       OX40N      89516.0      3
H-MC       XY25       88525.0      1
M-SC       SR         81493.0      1
H-LC       AR         77455.0      1
M-SC       X40        72347.0      1
H-LC       X200       69477.0      1
           X5K        66751.0      2
X-SC       XY24       65300.0      1
H-MC       AR         64061.0      1
L-SC       AR         59296.0      1
X-MC       X40N       58373.0      1
L-SC       XY24       57555.0      1
H-SC       MH         52159.0      1
L-MC       XR         51842.0      1
L-LC       XY25       44764.0      1
X-SC       OX40N      40330.0      2
H-LC       XY24       39561.0      1
X-LC       X40N       36786.0      1
H-MC       X40N       36356.0      1
           MH         30559.0      1
           X40        29385.0      1
X-MC       OX40N      15392.0      1
M-LC       XY25       10537.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
